referensi https://www.analyticsvidhya.com/blog/2021/04/forward-feature-selection-and-its-implementation/

In [1]:
#instal library mlxtend
# %pip install mlxtend

In [1]:
#libraries buat seleksi fitur
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif,SelectKBest 
from sklearn.feature_selection import mutual_info_classif

#libraries buat klasifikasi
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [2]:
#import dataset
df = pd.read_csv('nirsMangga.csv')

# separate dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(labels=['No', 'Mango Cultivars', 'Vit C (mg/100g)', 'TA (mg/100g)', 'SSC (oBrix)', 'label'], axis=1),
    df['label'],
    test_size=0.3,
    random_state=0)

x_train.head()

,999.9,1000.3,1000.7,1001.1,1001.4,1001.8,1002.2,1002.6,1003,1003.4,...,2478.7,2481.1,2483.5,2485.8,2488.2,2490.6,2493,2495.4,2497.8,2500.2
16,0.471459,0.471074,0.470934,0.470379,0.470260,0.469880,0.469497,0.469435,0.469454,0.468998,...,1.413537,1.415740,1.417568,1.419698,1.421711,1.423070,1.424394,1.426121,1.427552,1.428625
51,0.433239,0.432622,0.432626,0.432379,0.431620,0.430710,0.430836,0.430847,0.430188,0.429470,...,1.601232,1.602877,1.604524,1.605982,1.606778,1.607837,1.608756,1.609967,1.610900,1.611099
183,0.545045,0.544204,0.543792,0.543596,0.543338,0.542534,0.541493,0.541139,0.541308,0.540831,...,1.524657,1.525973,1.527454,1.529518,1.530097,1.530315,1.530254,1.531191,1.532366,1.533183
145,0.545846,0.544815,0.544524,0.544631,0.544169,0.543143,0.542535,0.542080,0.541842,0.541258,...,1.421962,1.422955,1.423717,1.424639,1.425080,1.425797,1.426503,1.427164,1.427838,1.428271
40,0.381048,0.380483,0.380541,0.380151,0.379599,0.379189,0.379009,0.378722,0.378309,0.377719,...,1.571125,1.572674,1.574303,1.576075,1.577273,1.577980,1.578561,1.579334,1.580042,1.581424


In [3]:
%%time

# determine the mutual information
mutual_info = mutual_info_classif(x_train, y_train)
mutual_info

CPU times: user 4.26 s, sys: 90.3 ms, total: 4.35 s
Wall time: 4.35 s


array([0.63194121, 0.63677335, 0.63393088, ..., 0.5091611 , 0.49290951,
       0.48903907])

In [4]:
x_train.columns

Index(['999.9', '1000.3', '1000.7', '1001.1', '1001.4', '1001.8', '1002.2',
       '1002.6', '1003', '1003.4',
       ...
       '2478.7', '2481.1', '2483.5', '2485.8', '2488.2', '2490.6', '2493',
       '2495.4', '2497.8', '2500.2'],
      dtype='object', length=1557)

In [5]:
# Create features score, features p_value, and features name
features_mi = pd.DataFrame(mutual_info)
features = pd.DataFrame(x_train.columns)
features_mi = pd.concat([features, features_mi],axis=1)

#reset index, supaya tidak ada nilai nan di dataframe yg dibikin
features_mi.reset_index(drop=True, inplace=True)
features.reset_index(drop=True, inplace=True)

# Assign the column name
features_mi.columns = ["Input_Features", "Mutual Information Value"]

# Print features score
print(features_mi.nlargest(300,columns="Mutual Information Value"))

    Input_Features  Mutual Information Value
318         1139.7                  0.709039
319         1140.2                  0.703579
322         1141.7                  0.702486
315         1138.2                  0.700575
316         1138.7                  0.699476
..             ...                       ...
455         1212.7                  0.598398
630         1320.8                  0.598360
983         1610.4                  0.598247
652         1335.8                  0.598080
653         1336.5                  0.597830

[300 rows x 2 columns]


# Seleksi Fitur Backward Selection dengan Random Forest Classifier

In [6]:
x_train_selected = x_train[features_mi.nlargest(300,columns="Mutual Information Value").iloc[0:300, 0]]
x_test_selected = x_test[features_mi.nlargest(300,columns="Mutual Information Value").iloc[0:300, 0]]

In [7]:
x_train_selected

,1139.7,1140.2,1141.7,1138.2,1138.7,1139.2,1140.7,1141.2,1142.2,1132.2,...,1333,1287.4,1207.6,1346.2,1007.3,1212.7,1320.8,1610.4,1335.8,1336.5
16,0.492063,0.493875,0.500545,0.485824,0.487906,0.489890,0.496089,0.498279,0.502916,0.464507,...,0.634095,0.556184,0.569706,0.665571,0.465651,0.568512,0.603703,0.842620,0.641125,0.642758
51,0.464010,0.466825,0.475396,0.454894,0.457810,0.460916,0.469729,0.472565,0.478432,0.426489,...,0.668614,0.555730,0.573913,0.715781,0.425145,0.572004,0.623792,0.985393,0.679198,0.681701
183,0.586312,0.589698,0.600224,0.575629,0.579215,0.582786,0.593261,0.596755,0.603650,0.540704,...,0.810916,0.692120,0.707730,0.855471,0.534954,0.706122,0.766246,1.073005,0.820799,0.823288
145,0.577506,0.580543,0.589776,0.568155,0.571318,0.574446,0.583619,0.586613,0.592968,0.538299,...,0.776353,0.669018,0.687148,0.818788,0.536735,0.685297,0.734829,1.030706,0.785930,0.788264
40,0.402640,0.405209,0.413083,0.395286,0.397766,0.400044,0.407849,0.410373,0.415739,0.370542,...,0.579554,0.480875,0.503912,0.621508,0.373607,0.501926,0.539874,0.869928,0.588804,0.591019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.543538,0.546104,0.554676,0.535210,0.538105,0.540964,0.549027,0.551875,0.557337,0.508815,...,0.730200,0.630338,0.643349,0.769883,0.504845,0.642064,0.691601,0.982645,0.739100,0.741312
67,0.473749,0.476239,0.484654,0.465488,0.468295,0.471059,0.479081,0.481982,0.487460,0.439093,...,0.661354,0.556828,0.573851,0.704487,0.439315,0.572048,0.619995,0.938691,0.671106,0.673352
117,0.503392,0.506553,0.516428,0.493610,0.496839,0.500089,0.509868,0.513116,0.519892,0.462708,...,0.740155,0.607688,0.629789,0.795169,0.461213,0.627599,0.687533,1.094887,0.752581,0.755331
47,0.459436,0.462290,0.471228,0.450392,0.453445,0.456400,0.465182,0.468146,0.474372,0.421441,...,0.665888,0.550905,0.568889,0.713774,0.420312,0.566776,0.620321,0.980873,0.676627,0.679137


In [8]:
# calling the linear regression model
lreg = RandomForestClassifier(n_estimators=100)
sfs1 = sfs(lreg, k_features=100, forward=False, verbose=2, scoring='accuracy', n_jobs=-1)

In [9]:
%%time

sfs1 = sfs1.fit(x_train_selected, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 292 out of 300 | elapsed:   27.4s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   27.7s finished

[2022-06-06 21:28:10] Features: 299/100 -- score: 0.8846153846153847[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 290 out of 299 | elapsed:   20.6s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done 299 out of 299 | elapsed:   20.9s finished

[2022-06-06 21:28:31] Features: 298/100 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 289 out of 298 | elapsed:   20.5s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done 298 out of 298 | elapsed:   20.9s

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 257 out of 277 | elapsed:   16.9s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done 277 out of 277 | elapsed:   17.8s finished

[2022-06-06 21:35:38] Features: 276/100 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 256 out of 276 | elapsed:   16.4s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done 276 out of 276 | elapsed:   17.1s finished

[2022-06-06 21:35:55] Features: 275/100 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 254 out of 275 | elapsed:   16.6s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done 275 out of 275 | elapsed:   17.2s finished

[2022-06-06 21:36:12] Features: 274/100 -- score: 0.876923076923077

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 223 out of 254 | elapsed:   11.6s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done 254 out of 254 | elapsed:   12.6s finished

[2022-06-06 21:41:21] Features: 253/100 -- score: 0.8846153846153847[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 221 out of 253 | elapsed:   11.5s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done 253 out of 253 | elapsed:   12.7s finished

[2022-06-06 21:41:34] Features: 252/100 -- score: 0.8923076923076924[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 220 out of 252 | elapsed:   11.0s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done 252 out of 252 | elapsed:   12.3s finished

[2022-06-06 21:41:46] Features: 251/100 -- score: 0.8769230769230

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 188 out of 231 | elapsed:    9.6s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done 231 out of 231 | elapsed:   11.6s finished

[2022-06-06 21:45:59] Features: 230/100 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 187 out of 230 | elapsed:    9.8s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done 230 out of 230 | elapsed:   12.0s finished

[2022-06-06 21:46:11] Features: 229/100 -- score: 0.8846153846153847[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 185 out of 229 | elapsed:   10.0s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done 229 out of 229 | elapsed:   11.7s finished

[2022-06-06 21:46:23] Features: 228/100 -- score: 0.87692307692307

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 154 out of 208 | elapsed:    8.5s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done 208 out of 208 | elapsed:   11.1s finished

[2022-06-06 21:50:19] Features: 207/100 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 152 out of 207 | elapsed:    9.0s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done 207 out of 207 | elapsed:   11.5s finished

[2022-06-06 21:50:30] Features: 206/100 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 151 out of 206 | elapsed:    8.3s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done 206 out of 206 | elapsed:   10.9s finished

[2022-06-06 21:50:41] Features: 205/100 -- score: 0.876923076923077

...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  85 out of 162 | elapsed:    5.8s remaining:    5.2s
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:   10.0s finished

[2022-06-06 21:58:31] Features: 161/100 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  83 out of 161 | elapsed:    6.1s remaining:    5.7s
[Parallel(n_jobs=-1)]: Done 161 out of 161 | elapsed:   10.6s finished

[2022-06-06 21:58:42] Features: 160/100 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  82 out of 160 | elapsed:    5.6s remaining:    5.3s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   10.2s finished

[2022-06-06 21:58:52] Features: 159/100 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concu

[Parallel(n_jobs=-1)]: Done  50 out of 139 | elapsed:    3.9s remaining:    6.9s
[Parallel(n_jobs=-1)]: Done 120 out of 139 | elapsed:    8.3s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done 139 out of 139 | elapsed:    9.6s finished

[2022-06-06 22:02:22] Features: 138/100 -- score: 0.876923076923077[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  49 out of 138 | elapsed:    4.0s remaining:    7.3s
[Parallel(n_jobs=-1)]: Done 119 out of 138 | elapsed:    8.5s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done 138 out of 138 | elapsed:    9.7s finished

[2022-06-06 22:02:32] Features: 137/100 -- score: 0.8692307692307694[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  47 out of 137 | elapsed:    3.6s remaining:    7.0s
[Parallel(n_jobs=-1)]: Done 116 out of 137 | elapsed:    8.1s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done 137 out of 137 | elapsed:    9.4s finished

[2022-06-

[Parallel(n_jobs=-1)]: Done  17 out of 117 | elapsed:    1.9s remaining:   11.4s
[Parallel(n_jobs=-1)]: Done  76 out of 117 | elapsed:    5.8s remaining:    3.1s
[Parallel(n_jobs=-1)]: Done 117 out of 117 | elapsed:    8.5s finished

[2022-06-06 22:05:45] Features: 116/100 -- score: 0.8692307692307694[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of 116 | elapsed:    2.0s remaining:   12.5s
[Parallel(n_jobs=-1)]: Done  75 out of 116 | elapsed:    6.1s remaining:    3.4s
[Parallel(n_jobs=-1)]: Done 116 out of 116 | elapsed:    9.0s finished

[2022-06-06 22:05:54] Features: 115/100 -- score: 0.8692307692307694[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 out of 115 | elapsed:    1.7s remaining:   12.1s
[Parallel(n_jobs=-1)]: Done  72 out of 115 | elapsed:    5.6s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done 115 out of 115 | elapsed:    8.4s finished

[2022-06

CPU times: user 30min 38s, sys: 25.7 s, total: 31min 4s
Wall time: 40min 15s


[Parallel(n_jobs=-1)]: Done 101 out of 101 | elapsed:    7.8s finished

[2022-06-06 22:07:57] Features: 100/100 -- score: 0.876923076923077

In [10]:
feat_names = list(sfs1.k_feature_names_)
print("Fitur yang diambil adalah: ")
print(feat_names)
len(feat_names)

Fitur yang diambil adalah: 
['1140.2', '1141.7', '1138.2', '1139.2', '1141.2', '1142.2', '1132.2', '1137.7', '1137.2', '1359.6', '1358.9', '1360.3', '1133.7', '1358.2', '1133.2', '1356.7', '1275.3', '1262.3', '1267.8', '1266.6', '1369.6', '1263.5', '1361.7', '1368.9', '1135.7', '1276.6', '1268.5', '1269.7', '1264.7', '1261.7', '1134.7', '1277.2', '1274.1', '1279.7', '1135.2', '1659.9', '1130.7', '1632.7', '1368.2', '1279.1', '1654.6', '1349.7', '1658.8', '1000.3', '1628.6', '1297.7', '1277.8', '1280.4', '1353.9', '1000.7', '1282.9', '1371.1', '1260.4', '1281', '1278.5', '1282.3', '1626.6', '1629.6', '1339.2', '1666.3', '1627.6', '1001.4', '1649.3', '1694.6', '1001.1', '1667.4', '1693.5', '1172.7', '1348.3', '1330.3', '1002.2', '1670.6', '1355.3', '1130.2', '1004.2', '1673.8', '1366', '1178', '1701.3', '1178.5', '1346.9', '1299', '1681.4', '1329.6', '1686.9', '1321.5', '1331', '1351.1', '1177.4', '1173.2', '1679.2', '1316.8', '1322.1', '1844', '1285.4', '1207', '1647.2', '1612.4', '1624

100

# Klasifikasi Random Forest Classifier

In [11]:
#buat n-fold cv
#cross validation 10-fold
cv = KFold(n_splits=10, random_state=1, shuffle=True)

## Testing Performa Model Random Forest Classifier

In [12]:
%%time
#tentukan metode scoring yang digunakan
scoring_rfe = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur dan trees yang digunakan dalam proses klasifikasi ini
#n_feat = [10, 20, 30]
n_feat = [100]
n_trees = [100]

for nfeat in n_feat:
    for ntrees in n_trees:        
        #ambil n fitur input hasil seleksi fitur Forward 
        x_train_selected = x_train[feat_names].iloc[:,0:nfeat]
        x_test_selected = x_test[feat_names].iloc[:,0:nfeat]

        #Create a Gaussian Classifier
        clf_rfe = RandomForestClassifier(n_estimators=ntrees)
        
        #Train the model using the training sets
        clf_rfe.fit(x_train_selected, y_train)
        y_pred_rfe=clf_rfe.predict(x_test_selected)
        
        #hitung score model dari data train
        scores_rfe = cross_validate(clf_rfe, x_train_selected, y_train, scoring=scoring_rfe, cv=cv, return_train_score=True)
        
        print("Akurasi model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_acc'].mean(), 2)))
        print("Akurasi model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.accuracy_score(y_test, y_pred_rfe), 2)))
        print("Precision model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_prec_micro'].mean(), 2)))
        print("Precision model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)))
        print("Recall model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_rec_micro'].mean(), 2)))
        print("Recall model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)))
        print(" ")

Akurasi model RFE data Train dengan 100 fitur dan 100 trees:1.0
Akurasi model RFE data Test dengan 100 fitur dan 100 trees:0.95
Precision model RFE data Train dengan 100 fitur dan 100 trees:1.0
Precision model RFE data Test dengan 100 fitur dan 100 trees:0.95
Recall model RFE data Train dengan 100 fitur dan 100 trees:1.0
Recall model RFE data Test dengan 100 fitur dan 100 trees:0.95
 
CPU times: user 1.44 s, sys: 16.1 ms, total: 1.46 s
Wall time: 1.46 s


In [13]:
%%time
#tentukan metode scoring yang digunakan
scoring_rfe = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur dan trees yang digunakan dalam proses klasifikasi ini
#n_feat = [10, 20, 30]
n_feat = [100]
n_trees = [150]

for nfeat in n_feat:
    for ntrees in n_trees:        
        #ambil n fitur input hasil seleksi fitur Forward 
        x_train_selected = x_train[feat_names].iloc[:,0:nfeat]
        x_test_selected = x_test[feat_names].iloc[:,0:nfeat]

        #Create a Gaussian Classifier
        clf_rfe = RandomForestClassifier(n_estimators=ntrees)
        
        #Train the model using the training sets
        clf_rfe.fit(x_train_selected, y_train)
        y_pred_rfe=clf_rfe.predict(x_test_selected)
        
        #hitung score model dari data train
        scores_rfe = cross_validate(clf_rfe, x_train_selected, y_train, scoring=scoring_rfe, cv=cv, return_train_score=True)
        
        print("Akurasi model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_acc'].mean(), 2)))
        print("Akurasi model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.accuracy_score(y_test, y_pred_rfe), 2)))
        print("Precision model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_prec_micro'].mean(), 2)))
        print("Precision model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)))
        print("Recall model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_rec_micro'].mean(), 2)))
        print("Recall model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)))
        print(" ")

Akurasi model RFE data Train dengan 100 fitur dan 150 trees:1.0
Akurasi model RFE data Test dengan 100 fitur dan 150 trees:0.93
Precision model RFE data Train dengan 100 fitur dan 150 trees:1.0
Precision model RFE data Test dengan 100 fitur dan 150 trees:0.93
Recall model RFE data Train dengan 100 fitur dan 150 trees:1.0
Recall model RFE data Test dengan 100 fitur dan 150 trees:0.93
 
CPU times: user 2.08 s, sys: 35.8 ms, total: 2.12 s
Wall time: 2.11 s


In [14]:
%%time
#tentukan metode scoring yang digunakan
scoring_rfe = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur dan trees yang digunakan dalam proses klasifikasi ini
#n_feat = [10, 20, 30]
n_feat = [100]
n_trees = [200]

for nfeat in n_feat:
    for ntrees in n_trees:        
        #ambil n fitur input hasil seleksi fitur Forward 
        x_train_selected = x_train[feat_names].iloc[:,0:nfeat]
        x_test_selected = x_test[feat_names].iloc[:,0:nfeat]

        #Create a Gaussian Classifier
        clf_rfe = RandomForestClassifier(n_estimators=ntrees)
        
        #Train the model using the training sets
        clf_rfe.fit(x_train_selected, y_train)
        y_pred_rfe=clf_rfe.predict(x_test_selected)
        
        #hitung score model dari data train
        scores_rfe = cross_validate(clf_rfe, x_train_selected, y_train, scoring=scoring_rfe, cv=cv, return_train_score=True)
        
        print("Akurasi model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_acc'].mean(), 2)))
        print("Akurasi model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.accuracy_score(y_test, y_pred_rfe), 2)))
        print("Precision model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_prec_micro'].mean(), 2)))
        print("Precision model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.precision_score(y_test, y_pred_rfe, average='micro'), 2)))
        print("Recall model RFE data Train dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(scores_rfe['train_rec_micro'].mean(), 2)))
        print("Recall model RFE data Test dengan " + str(nfeat) + " fitur dan " + str(ntrees) + " trees:" 
              + str(round(metrics.recall_score(y_test, y_pred_rfe, average='micro'), 2)))
        print(" ")

Akurasi model RFE data Train dengan 100 fitur dan 200 trees:1.0
Akurasi model RFE data Test dengan 100 fitur dan 200 trees:0.96
Precision model RFE data Train dengan 100 fitur dan 200 trees:1.0
Precision model RFE data Test dengan 100 fitur dan 200 trees:0.96
Recall model RFE data Train dengan 100 fitur dan 200 trees:1.0
Recall model RFE data Test dengan 100 fitur dan 200 trees:0.96
 
CPU times: user 2.78 s, sys: 32.2 ms, total: 2.82 s
Wall time: 2.81 s


# Klasifikasi Support Vector Machine

Untuk train dan test set, serta cv-nya ambil dari yang Random Forest Classifier, sama aja 

## Testing Performa Model Support Vector Machine

In [15]:
%%time
#tentukan metode scoring yang digunakan
scoring_svm = {'acc': 'accuracy',
               'prec_micro': 'precision_micro',
               'rec_micro': 'recall_micro'}

#tentukan total fitur yang digunakan dalam proses klasifikasi SVM ini
#n_feat = [10, 20, 30]
n_feat = [100]

for nfeat in n_feat:     
    #ambil n fitur input hasil seleksi fitur Forward 
    x_train_selected = x_train[feat_names].iloc[:,0:nfeat]
    x_test_selected = x_test[feat_names].iloc[:,0:nfeat]

    #Create a Support Vector Classifier
    clf_svm = svm.SVC()

    #Train the model using the training sets
    clf_svm.fit(x_train_selected, y_train)
    y_pred_svm = clf_svm.predict(x_test_selected)

    #hitung score model dari data train
    scores_svm = cross_validate(clf_svm, x_train_selected, y_train, scoring=scoring_svm, cv=cv, return_train_score=True)

    print("akurasi model SVM data Train dengan " + str(nfeat) + " fitur: " 
          + str(round(scores_svm['train_acc'].mean(), 2)))
    print("akurasi model SVM data Test dengan " + str(nfeat) + " fitur: " 
          + str(round(metrics.accuracy_score(y_test, y_pred_svm), 2)))
    print("Precision model SVM data Train dengan " + str(nfeat) + " fitur:"
          + str(round(scores_svm['train_prec_micro'].mean(), 2)))
    print("Precision model SVM data Test dengan " + str(nfeat) + " fitur:"
          + str(round(metrics.precision_score(y_test, y_pred_svm, average='micro'), 2)))
    print("Recall model SVM data Train dengan " + str(nfeat) + " fitur:"
          + str(round(scores_svm['train_rec_micro'].mean(), 2)))
    print("Recall model SVM data Test dengan " + str(nfeat) + " fitur:"
          + str(round(metrics.recall_score(y_test, y_pred_svm, average='micro'), 2)))
    print(" ")

akurasi model SVM data Train dengan 100 fitur: 0.74
akurasi model SVM data Test dengan 100 fitur: 0.66
Precision model SVM data Train dengan 100 fitur:0.74
Precision model SVM data Test dengan 100 fitur:0.66
Recall model SVM data Train dengan 100 fitur:0.74
Recall model SVM data Test dengan 100 fitur:0.66
 
CPU times: user 126 ms, sys: 4.02 ms, total: 130 ms
Wall time: 125 ms
